<a href="https://colab.research.google.com/github/SayoojDinesan/KidneyCancerCTSCAN/blob/main/Sayooj_Dinesan_Kidney_Problem_Identifier_CT_Scan_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
train_data_dir = "/content/drive/MyDrive/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/Training_Data"
val_data_dir = "/content/drive/MyDrive/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/Validation_Data"


In [ ]:
img_width = 512
img_height = 512

train_samples = 280
val_samples = 80

epochs = 20
batch_size = 100

'''
import matplotlib.pyplot as plt
for img in val_data_dir:
  image = plt.imread(f"{val_data_dir}/{img}")
  plt.imshow(image)
'''

'\nimport matplotlib.pyplot as plt\nfor img in val_data_dir:\n  image = plt.imread(f"{val_data_dir}/{img}")\n  plt.imshow(image)\n'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode = "binary"
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode = "binary"
)

Found 140 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [ ]:
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

model_top = Sequential()
model_top.add(GlobalAveragePooling2D(input_shape=base_model.output_shape[1:], data_format=None)),
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=model_top(base_model.output))

model.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.0), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=train_samples // batch_size,
            epochs=epochs,
            validation_data=val_generator,
            validation_steps=val_samples // batch_size)

<ipython-input-6-c7c9127d4ac4>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20


In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
import matplotlib.pyplot as plt

'''
img_path='/content/drive/MyDrive/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/Testing_Data/Cyst_Testing/Cyst-(100).jpg' #change to location of abd x-ray
img_path2='/content/drive/MyDrive/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/Testing_Data/Normal_Testing/Normal-(91).jpg'  #change to location of chest x-ray
img = load_img(img_path, target_size=(img_width, img_height))
img2 = load_img(img_path2, target_size=(img_width, img_height))
plt.imshow(img)
plt.show()

img = img_to_array(img)
x = np.expand_dims(img, axis=0) * 1./255
score = model.predict(x)
print('Predicted:', score, 'Chest X-ray' if score < 0.5 else 'Abd X-ray')

plt.imshow(img2)
plt.show()

img = img_to_array(img2)
x = np.expand_dims(img2, axis=0) * 1./255
score2 = model.predict(x)
print('Predicted:', score2, 'Chest X-ray' if score2 < 0.5 else 'Abd X-ray')
'''
# Set the path to your test data directory
test_data_dir = '/content/drive/MyDrive/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/Testing_Data'

# Use the ImageDataGenerator to rescale the test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Generate the test data from the directory
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=5,
    class_mode='categorical',
    shuffle=False)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate_generator(
    test_generator,
    steps=test_generator.samples // 5)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


Found 30 images belonging to 3 classes.


<ipython-input-12-a2a01c11f7af>:42: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy = model.evaluate_generator(


Test Loss: 1.2161352634429932
Test Accuracy: 0.6666666865348816


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess the image
img_path = '/content/drive/MyDrive/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/CT_KIDNEY_DATASET_Normal_Cyst_Tumor_Stone/Training_Data/Tumor_Training/Tumor-(36).jpg'
img = image.load_img(img_path, target_size=(img_width, img_height))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

# Make predictions
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Get the class label
class_labels = ['Tumor', 'Normal', 'Cyst']  # Replace with your actual class labels
predicted_label = class_labels[predicted_class[0]]

# Print the predicted label
print("Predicted category: ", predicted_label)


1/1 [==============================] - 1s 1s/step
Predicted category:  Cyst
